In [93]:
# Generalized + Individualized Approach
# Train & Test on Distinct Groups of Participants
# Also Use Small % of Test Participant's Data (Chronological Split)

# Evaluate "generalizability" of each train participant's session for particular test participant

In [94]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools
import json
pd.options.mode.chained_assignment = None

In [95]:
# Key Features: op_num_people, of_pose_distance, of_gaze_distance, of_confidence, ros_mistakes_session, ros_ts_robot_talked, timestamp
use_key_features = False

# Leave One Out Run: test on each participant & train on remaining
leave_one_out_run = True

# Number of Estimators for Bagging
num_estimators = 10

# Sample Percentage for Bagging
sample_percentage = 0.75

# Threshold to Use
threshold_to_use = 0.75
use_all_if_lower_than_threshold = True

# Percentage of Test Participant Data to Use
test_percentage_to_use = 0.3

# File Name for Model Results
output_file = 'multi4_session_af_xgb_results_75.csv'

metric_keys = ['accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1']

# 1 -- Load Data

In [96]:
# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

In [97]:
data = [data5, data7, data9, data11, data12, data17, data18]
part = [5, 7, 9, 11, 12, 17, 18]

In [98]:
remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',
'ros_game_correct',
'ros_game_incorrect',
'ros_game_start',
'ros_mistake_made',
]

for i, d in enumerate(data):
    data[i] = d.drop(columns=remove)

In [99]:
# Remove rows where engagment NaN
for i,d in enumerate(data):
    data[i] = d[np.isfinite(d['engagement'])]
    
# Remove rows where engagment is -1
for i,d in enumerate(data):
    data[i] = d[d['engagement']>=0]

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [100]:
# Main Columns
basic_cols = []
for i in data[0].columns:
    if 'of_' not in i and 'op_' not in i and 'ros_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
participant
session_num
timestamp


In [101]:
# Open Face Columns

of_cols = []
for i in data[0].columns:
    if ('of_' in i or 'op_' in i) and '_change' not in i and '_var' not in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU02_c
of_AU04_c
of_AU05_c
of_AU06_c
of_AU07_c
of_AU09_c
of_AU10_c
of_AU12_c
of_AU14_c
of_AU15_c
of_AU17_c
of_AU20_c
of_AU23_c
of_AU25_c
of_AU26_c
of_AU28_c
of_AU45_c
of_confidence
of_gaze_0_x
of_gaze_0_y
of_gaze_0_z
of_gaze_1_x
of_gaze_1_y
of_gaze_1_z
of_gaze_angle_x
of_gaze_angle_y
of_gaze_distance
of_gaze_distance_x
of_gaze_distance_y
of_pose_Rx
of_pose_Ry
of_pose_Rz
of_pose_Tx
of_pose_Ty
of_pose_Tz
of_pose_distance
of_success
of_ts_success
op_num_people


In [102]:
# Audio Columns

a_cols = []
for i in data[0].columns:
    if 'a_' in i and '_change' not in i and '_var' not in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_intensity
a_mfcc_0
a_mfcc_1
a_pitch_frequency
a_pitch_strength


In [103]:
# ROS Columns

ros_cols = []
for i in data[0].columns:
    if 'ros_' in i and '_change' not in i and '_var' not in i:
        ros_cols.append(i)
        
ros_cols = sorted(ros_cols)
for i in ros_cols:
    print(i)

ros_aptitude
ros_diff_1
ros_diff_2
ros_diff_3
ros_diff_4
ros_diff_5
ros_games_session
ros_in_game
ros_mistakes_game
ros_mistakes_session
ros_skill_EM
ros_skill_NC
ros_skill_OS
ros_ts_attempt
ros_ts_game_start
ros_ts_robot_talked


In [104]:
# For Window Only:
non_window_features = []
window_features = []
for i in data[0].columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_c_change
of_AU02_c_change
of_AU04_c_change
of_AU05_c_change
of_AU06_c_change
of_AU07_c_change
of_AU09_c_change
of_AU10_c_change
of_AU12_c_change
of_AU14_c_change
of_AU15_c_change
of_AU17_c_change
of_AU20_c_change
of_AU23_c_change
of_AU25_c_change
of_AU26_c_change
of_AU28_c_change
of_AU45_c_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_gaze_distance_x_var
of_gaze_distance_y_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
of_ts_success_var
op_num_people_change
ros_aptitude_var


In [105]:
# Columns where NaNs filled with max value
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance']

In [106]:
# Key Features (from feature analysis) 
# Note: Timestamp automatically included

key_features = [
'op_num_people',
'of_pose_distance',
'of_gaze_distance',
'of_confidence',
'ros_mistakes_session',
'ros_ts_robot_talked']

### Filter Feature Set (Optional)

In [107]:
# always include basic_cols, add desired group of features 
# features_to_keep = all_data.columns

if use_key_features:
    features_to_keep = basic_cols + key_features
else:
    features_to_keep = basic_cols + of_cols + ros_cols + a_cols + window_features

for i,d in enumerate(data):
    data[i] = d[features_to_keep]

# 3 -- Scenario Based Modeling

- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [108]:
# Create Separate Models for Different Scenarios

# for i,d in enumerate(data):
#     # Open Face Success/Failure
#     data[i] = d.loc[d['of_success']==1]
#     data[i] = d.loc[d['of_success']==0]
    
#     # Robot Talking/Not Talking
#     data[i] = d.loc[d['ros_ts_robot_talked']==0]
#     data[i] = d.loc[d['ros_ts_robot_talked']>0]
    
#     # First 10 Min/After 10 Min
#     data[i] = d.loc[d['timestamp']<=(10*60)]
#     data[i] = d.loc[d['timestamp']>(10*60)]

# 4 -- Modeling

In [109]:
# Helper Function: Split a Participant's Data Chronologically 

def part_split(split, split_size):
    split = split.sort_values(['session_num', 'timestamp'], ascending=[True, True])
    bogus = split['engagement']

    split_train, split_test, bogus1, bogus2 = train_test_split(split, bogus, test_size=split_size, shuffle=False)
    return split_test, split_train

In [110]:
# Function: Formulate Train-Test Split 
# Includes Preprocessing & Shuffle 

# train_part: an array of participants 
# test_part: an array of participants
# split_size: how much of each test participant's data to use for validation
def split(train_part, test_part, split_size):
    train_data = []
    for p in train_part:
        i = part.index(p)
        train_data.append(data[i].copy())
    
    val_data = []
    test_data = []
    for p in test_part:
        i = part.index(p)
        if split_size > 0:
            p_test, p_train = part_split(data[i].copy(), split_size)
            val_data.append(p_train)
            test_data.append(p_test)
        else:
            test_data.append(data[i].copy())
        
    train = pd.concat(train_data, ignore_index=True, sort=True)
    val = pd.concat(val_data, ignore_index=True, sort=True)
    test = pd.concat(test_data, ignore_index=True, sort=True)
    
    # Sort Training & Validation Data 
    train = train.sample(frac=1)
    train = train.sample(frac=1)
    train = train.sample(frac=1)
    val = val.sample(frac=1)
    val = val.sample(frac=1)
    val = val.sample(frac=1)

    # Preprocessing: Standardization
    # x' = ( x - mean(x) ) / ( stdev(x) )
    # => x' is z-score, NaN's filled with min_val
    for c in train.columns:
        if (c in ['engagement', 'participant', 'session_num']):
            continue
            
        mean = np.nanmean(train[c])
        std = np.nanstd(train[c])

        if std == 0:
            train[c] = (train[c]-mean)
            val[c] = (val[c]-mean)
            test[c] = (test[c]-mean)
        else:
            train[c] = (train[c]-mean)/(std)
            val[c] = (val[c]-mean)/(std)
            test[c] = (test[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(train[c])

            train[c] = train[c].fillna(min_val)
            val[c] = val[c].fillna(min_val)
            test[c] = test[c].fillna(min_val)
        else:
            max_val = np.nanmax(train[c])

            train[c] = train[c].fillna(max_val)
            val[c] = val[c].fillna(max_val)
            test[c] = test[c].fillna(max_val) 

    return train, val, test

In [111]:
# MODEL HERE
# Multiple Participants Approach 4: 
# Approach 4: Use Small % of Test Participant's Data (Chronological Split)
# Iterate over each other participant & select most "similar" to test participant

# Inputs: X_train, y_train, X_val, y_val, X_test, y_test

# Output: Accuracy Metrics as Dictionary 

# IMPORTS HERE
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn import neighbors

def model(X_train, y_train, X_test, y_test, is_verbose, use_bagging):    
    # Shuffle X_train & y_train just in case
    idx = np.random.permutation(len(X_train))
    X_train = X_train.iloc[idx]
    y_train = y_train.iloc[idx]
    X_train = X_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    
    # Model Here: must create pred & scores arrays
    if use_bagging:
        clf = BaggingClassifier(XGBClassifier(n_estimators=100), n_estimators=num_estimators, max_samples=sample_percentage)
        clf.fit(X_train.values, y_train.values)
        scores = clf.predict_proba(X_test.values)[:,1]
        pred = [round(value) for value in scores]   
    else:
        clf = XGBClassifier(n_estimators=100)
        clf.fit(X_train.values, y_train.values)
        scores = clf.predict(X_test.values)
        pred = [round(value) for value in scores] 

    # Evaluation
    results = {}
    accuracy = accuracy_score(y_test, pred)
    results['accuracy'] = accuracy
    
    if is_verbose:
        try:
            auc = roc_auc_score(y_test, scores)
        except:
            auc = np.nan
        results['auc'] = auc

        precision = metrics.precision_score(y_test, pred, average=None)
        recall = metrics.recall_score(y_test, pred, average=None)
        f1 = metrics.f1_score(y_test, pred, average=None)

        results['precision_0'], results['precision_1'] = precision[0], precision[1]
        results['recall_0'], results['recall_1'] = recall[0], recall[1]
        results['f1_0'], results['f1_1'] = f1[0], f1[1]
    
    return results

In [112]:
all_results = []
for n in range(1, len(part)):    
    if leave_one_out_run:
        if n != len(part)-1:
            continue
    
    comb = combinations(part, n)
    print()
    
    for c in comb:
        print()
        new_results = {}
        for m in metric_keys:
            new_results[m] = []
        
        # Determine Train & Test Data
        train_part = [x for x in c]
        test_part = [p for p in part if p not in c]

        train, val, test = split(train_part, test_part, test_percentage_to_use)
        new_train = train.append(val)
        new_train = new_train.reset_index(drop=True)

        # Evaluate "generalizability" of each train participant's session
        session_scores = {}
        for p in train_part:
            part_data = new_train[new_train['participant']==p]
            session_scores[p] = {}
            for s in part_data['session_num'].unique():
                curr_train = new_train[(new_train['participant']==p) & (new_train['session_num']==s)]
                curr_test = new_train[(new_train['participant']!=p) | (new_train['session_num']!=s)]
                
                curr_X_train = curr_train.drop(columns=['engagement', 'participant', 'session_num'])
                curr_X_test = curr_test.drop(columns=['engagement', 'participant', 'session_num'])
                curr_y_train = curr_train['engagement']
                curr_y_test = curr_test['engagement']
                
                curr_results = model(curr_X_train, curr_y_train, curr_X_test, curr_y_test, False, False)
                session_scores[p][s] = curr_results['accuracy']
                print("Part: ", p, "Session: ", s, "Accuracy: ", curr_results['accuracy'])
        
        # Create Final Training Data based on session scores
        final_train_data = pd.DataFrame(columns=train.columns)
        for p in train_part:
            part_data = new_train[new_train['participant']==p]
            for s in part_data['session_num'].unique():
                if session_scores[p][s] > threshold_to_use:
                    curr_data = new_train[(new_train['participant']==p) & (new_train['session_num']==s)]
                    final_train_data.append(curr_data)
        
        # Always include validation data
        if len(final_train_data)==0:
            final_train_data = new_train
        else:
            final_train_data.append(val)
        
        X_train = final_train_data.drop(columns=['engagement', 'participant', 'session_num'])
        X_test = test.drop(columns=['engagement', 'participant', 'session_num'])
        y_train = final_train_data['engagement']
        y_test = test['engagement']
        
        final_results = model(X_train, y_train, X_test, y_test, True, True)
        print(final_results)
        final_results['train_participants'] = ", ".join(str(x) for x in train_part)
        final_results['test_participants'] = ", ".join(str(x) for x in test_part)
        all_results.append(final_results)



Part:  5 Session:  7.0 Accuracy:  0.6453312676104296
Part:  5 Session:  2.0 Accuracy:  0.6574995538256265
Part:  5 Session:  13.0 Accuracy:  0.6813807655982779
Part:  5 Session:  8.0 Accuracy:  0.7686762272949366
Part:  5 Session:  10.0 Accuracy:  0.6594126167947209
Part:  7 Session:  5.0 Accuracy:  0.7848773230273525
Part:  7 Session:  29.0 Accuracy:  0.7737410409156376
Part:  7 Session:  8.0 Accuracy:  0.7480121117506283
Part:  7 Session:  6.0 Accuracy:  0.6856304886191712
Part:  7 Session:  16.0 Accuracy:  0.6288637889565963
Part:  7 Session:  7.0 Accuracy:  0.6978182947610533
Part:  7 Session:  18.0 Accuracy:  0.6905790546081721
Part:  7 Session:  17.0 Accuracy:  0.6546783062848917
Part:  9 Session:  15.0 Accuracy:  0.7484585189432365
Part:  9 Session:  4.0 Accuracy:  0.751000807593112
Part:  9 Session:  3.1 Accuracy:  0.6842332321070573
Part:  9 Session:  3.2 Accuracy:  0.7892597212624853
Part:  9 Session:  9.0 Accuracy:  0.6911807720271772
Part:  11 Session:  11.0 Accuracy:  0.

Part:  7 Session:  8.0 Accuracy:  0.7661851164565304
Part:  7 Session:  5.0 Accuracy:  0.7816027489940534
Part:  7 Session:  16.0 Accuracy:  0.6279337479811691
Part:  9 Session:  3.2 Accuracy:  0.7905729223396144
Part:  9 Session:  15.0 Accuracy:  0.7486129199150804
Part:  9 Session:  4.0 Accuracy:  0.7115138106024689
Part:  9 Session:  3.1 Accuracy:  0.6865455975649378
Part:  9 Session:  9.0 Accuracy:  0.6899746621621622
Part:  12 Session:  3.0 Accuracy:  0.7853467542142375
Part:  12 Session:  10.0 Accuracy:  0.7591817316841104
Part:  12 Session:  2.2 Accuracy:  0.7045359276207775
Part:  12 Session:  6.0 Accuracy:  0.8057629632184304
Part:  12 Session:  8.0 Accuracy:  0.7623598870539916
Part:  17 Session:  3.0 Accuracy:  0.6797236639079814
Part:  17 Session:  5.0 Accuracy:  0.740770660690398
Part:  17 Session:  2.0 Accuracy:  0.696690221344695
Part:  17 Session:  6.0 Accuracy:  0.6763322750963449
Part:  18 Session:  7.0 Accuracy:  0.6470937632583793
Part:  18 Session:  19.0 Accuracy: 

Part:  11 Session:  11.0 Accuracy:  0.7283954713225231
Part:  11 Session:  9.0 Accuracy:  0.7642583614240955
Part:  11 Session:  16.2 Accuracy:  0.7218037536213628
Part:  11 Session:  19.0 Accuracy:  0.6691868124491286
Part:  11 Session:  8.0 Accuracy:  0.6496674242235042
Part:  12 Session:  6.0 Accuracy:  0.8026049325658586
Part:  12 Session:  3.0 Accuracy:  0.7806693960241422
Part:  12 Session:  2.2 Accuracy:  0.6977393198390212
Part:  12 Session:  10.0 Accuracy:  0.7569478981396407
Part:  12 Session:  8.0 Accuracy:  0.7627884864003737
Part:  17 Session:  5.0 Accuracy:  0.737534385712472
Part:  17 Session:  2.0 Accuracy:  0.6961746512628798
Part:  17 Session:  3.0 Accuracy:  0.677000862606438
Part:  17 Session:  6.0 Accuracy:  0.6655686962566933
Part:  18 Session:  9.0 Accuracy:  0.7342030952799913
Part:  18 Session:  17.0 Accuracy:  0.6745906710108467
Part:  18 Session:  19.0 Accuracy:  0.7183830320622774
Part:  18 Session:  20.0 Accuracy:  0.6847268599458112
Part:  18 Session:  4.0

In [113]:
end_results = pd.DataFrame(columns=['train_participants', 'test_participants', 'accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1'])
end_results = end_results.append(all_results, ignore_index=True, sort=True)

In [114]:
end_results.head()

,accuracy,auc,f1_0,f1_1,precision_0,precision_1,recall_0,recall_1,test_participants,train_participants
0,0.768249,0.823476,0.378327,0.857578,0.633758,0.785088,0.269648,0.944818,18,"5, 7, 9, 11, 12, 17"
1,0.710513,0.882482,0.613613,0.768555,0.866010,0.654325,0.475135,0.931104,17,"5, 7, 9, 11, 12, 18"
2,0.865413,0.943266,0.859123,0.871166,0.887298,0.846582,0.832683,0.897220,12,"5, 7, 9, 11, 17, 18"
3,0.791685,0.814003,0.429365,0.872586,0.894215,0.781836,0.282507,0.987169,11,"5, 7, 9, 12, 17, 18"
4,0.854855,0.956195,0.877579,0.821773,0.940247,0.749087,0.822743,0.910081,9,"5, 7, 11, 12, 17, 18"


In [115]:
end_results.to_csv(output_file, index=False)